## 13.1 Dagger-Hilt - Podstawy

Wv tej aplikacji zapoznamy się z podstawami wykorzystania `Dagger-Hilt` w aplikacji. Jak zobaczymy zastosowanie biblioteki `Hilt` mocno upraszcza całą procedurę *wstrzyknięć*. Wykorzystamy `Retrofit2`, `LoggingInterceptor` i architekturę **MVVM**. Jako backend ponownie wybierzemy https://jsonplaceholder.typicode.com/.

Rozpocznijmy od dodania odpowiednich zależności

In [ ]:
plugins {
    id 'com.android.application'
    id 'com.google.dagger.hilt.android'
}
...
dependencies {

    implementation "com.google.dagger:hilt-android:2.44"
    annotationProcessor 'androidx.hilt:hilt-compiler:1.0.0'
    annotationProcessor "com.google.dagger:hilt-android-compiler:2.44"

    implementation 'com.squareup.retrofit2:retrofit:2.9.0'
    implementation 'com.google.code.gson:gson:2.9.1'
    implementation 'com.squareup.retrofit2:converter-gson:2.9.0'
    implementation 'com.squareup.okhttp3:logging-interceptor:4.10.0'
}

Do bloku `plugins` pliku `build.gradle(Project)` dodajemy

In [ ]:
id 'com.google.dagger.hilt.android' version '2.44' apply false

### Szkielet projektu

Naszą aplikację podzielimy na kilka pakietów
- `data` - model danych + dwa pakiety
    - `remote` zawierający interfejs `PlaceholderApi`
    - `repository` zawierający implementację repozytorium aplikacji
- `di` - przeznaczony do *dependency injection* - tutaj dodamy moduły
- `ui` - elementy interfejsu użytkownika
- w głównym pakiecie znajduje się klasa `MyApp` rozszerzająca `Application`

Rozpocznijmy od interfejsu `PlaceholderApi` w pakiecie `data/remote`. Jest on identyczny jak w przykładzie 11.1

In [ ]:
public interface PlaceholderApi {
    @GET("posts")
    Call<List<Post>> getPosts();
}

Do pakietu `data` dodajmy model danych (identyczny jak w 11.1)

In [ ]:
public class Post {
    private int userId;
    private int id;
    private String title;

    @SerializedName("body")
    String content;

    public int getUserId() {
        return userId;
    }

    public int getId() {
        return id;
    }

    public String getTitle() {
        return title;
    }

    public String getContent() {
        return content;
}
}

Do pakietu `data/repository` dodajmy klasę `AppRepository`

In [ ]:
public class AppRepository {
    private final PlaceholderApi api;

    public AppRepository(PlaceholderApi api) {
        this.api = api;
    }

    public Call<List<Post>> getPosts() {
        return api.getPosts();
    }
}

Do pakietu `di` dodajmy nasz moduł `AppModule`, tutaj zdefiniujemy instancję `Retrofit`. Chcemy posiadać jeden moduł o czasie życia aplikacji (powiązany z kontekstem `Application`), więc będzie to `object`

In [ ]:
@Module
public class AppModule {}

Drugą adnotacją którą zastosujemy jest `@InstallIn` - deklaruje do których komponentów opisana klasa/obiekt powinna zostać dodana podczas generacji obiektów przez `Hilt`.

In [ ]:
@Module
@InstallIn(SingletonComponent.class)
public class AppModule {}

Komponent podany jako argument `@InstallIn` decyduje o czasie życia zależności dodanych w module. Niektóre przekłady
- `SingletonComponent` - czas życia aplikacji (poprzednio nazywany `ApplicationComponent`)
- `ActivityComponent` - czas życia aktywności
- `ViewModelComponent` - czas życia `ViewModel`
- `ServiceComponent` - czas życia serwisu
- `ActivityRetainedComponent` - czas życia aktywności + zmiana konfiguracji (rotacja)

Dodajmy metodę `providePlaceholderApi`, gdzie utworzymy instancję `Retrofit` i dodamy `LoggingInterceptor`

In [ ]:
@Provides
@Singleton
PlaceholderApi providePlaceholderApi(){
    HttpLoggingInterceptor interceptor = new HttpLoggingInterceptor();
    interceptor.setLevel(HttpLoggingInterceptor.Level.BODY);
    OkHttpClient client = new OkHttpClient.Builder()
            .addInterceptor(interceptor)
            .build();
    return new Retrofit.Builder()
            .baseUrl("https://jsonplaceholder.typicode.com/")
            .addConverterFactory(GsonConverterFactory.create())
            .client(client)
            .build().create(PlaceholderApi.class);
}

Jedynym nowym elementem jest adnotacja `@Singleton` - jeżeli `PlaceholderApi` zostanie wstrzyknięty do kilku klas, będzie to ta sama instancja. Bez tej adnotacji, przy wstrzykiwaniu do kilku klas, za każdym razem tworzona jest nowa instancja.

Dodajmy drugą metodę dostarczającą repozytorium.

In [ ]:
@Provides
@Singleton
AppRepository provideRepository(PlaceholderApi api){
    return new AppRepositoryImpl(api);
}

Dodajmy `AppViewModel`, wykorzystamy adnotację `HiltViewModel`, który pozwala bibliotece `Hilt` wykonać wstrzyknięcia do `ViewModel`, oraz wstrzyknąć sam `ViewModel` (jak i zastosować w kotlinie `by viewmodel()` lub `ViewModelProvider` w Javie) - dzięki tej adnotacji jest to o wiele łatwiejszev niż w przypadku zastosowania biblioteki `Dagger2`.

Wykorzystamy adnotację `@Inject` do wstrzyknięcia przez konstruktor repozytorium do `AppViewModel`.

In [ ]:
@HiltViewModel
public class AppViewModel extends ViewModel {
        private final AppRepository repository;
    
    @Inject
    public AppViewModel(AppRepository repository) {
        this.repository = repository;
    }


Dodajmy listę wszystkich postów.

In [ ]:
private final MutableLiveData<List<Post>> posts = new MutableLiveData<>();

public LiveData<List<Post>> getPosts() {
        return posts;
    }

Na koniec dodajmy metodę `getPosts` wykorzystując `Coroutines` i `viewModelScope`

In [ ]:
public void getAllPosts(){
    Call<List<Post>> responseCall = repository.getPosts();

    responseCall.enqueue(new Callback<List<Post>>() {
        @Override
        public void onResponse(
            @NonNull Call<List<Post>> call, 
            @NonNull Response<List<Post>> response) {
            if (response.isSuccessful()){
                List<Post> postsResponse = response.body();
                if (postsResponse != null)
                    posts.postValue(postsResponse);
            }
        }

        @Override
        public void onFailure(
            @NonNull Call<List<Post>> call, 
            @NonNull Throwable t) {
            Log.e("TAG", "error: " + t.getMessage());
        }
    });
}

Jak widzimy, poza zastosowaniem kilku adnotacji w samym kodzie nic się nie zmienia.

Wymaganym jest również dodanie klasy rozszerzającej klasę `Application`, z adnotacją `HiltAndroidApp`

In [ ]:
@HiltAndroidApp
public class MyApp extends Application {
}

Dzięki temu `Hilt` *wie* że w tej klasie może tworzyć komponenty `Dagger2`. Adnotacja `@HiltAndroidApp` wyzwala generowanie komponentów `Dagger2` przez `Hilt`. Tutaj musimy jeszcze dokonać zmiany w pliku `AndroidManifest` i dodać `name` o nazwie klasy rozszerzającej `Application`

In [ ]:
<application
    android:name=".MyApp"
    ...

Ta klasa może zostać wykorzystana w momencie gdy musimy dostarczyć `applicationContext` (np. do `ROOM`) - `Hilt` automatycznie wygeneruje odpowiedni kod.

W aktywności głównej dodajemy `ViewModel`

In [ ]:
AppViewModel viewModel = new ViewModelProvider(this).get(AppViewModel.class);

W samej aktywności musimy dodać adnotację `@AndroidEntryPoint` - wskazuje punkt wejściowy aplikacji dla `Hilt`

In [ ]:
@AndroidEntryPoint
public class MainActivity extends AppCompatActivity {

W metodzie `onCreate` inicjujemy połączenie z serwerem.

In [ ]:
viewModel.getAllPosts();

W pozostałych elementach nie ma żadnych nowych elementów.

In [ ]:
viewModel.getPosts().observe(this, posts -> {
    StringBuilder content = new StringBuilder();
    posts.forEach( post ->
            content
                    .append("id: ").append(post.getId()).append("\n")
                    .append("UserId: ").append(post.getUserId()).append("\n")
                    .append("title: ").append(post.getTitle()).append("\n")
                    .append("body: ").append(post.getContent()).append("\n\n")
    );
    textView.setText(content.toString());
});